In [1]:
!pip3 install transformers==3.4.0
!pip3 install emoji
!pip3 install keras
!pip3 install tensorflow

  Using cached transformers-3.4.0-py3-none-any.whl (1.3 MB)
  Using cached sentencepiece-0.1.97-cp310-cp310-macosx_10_9_x86_64.whl (1.2 MB)
  Using cached sacremoses-0.0.53-py3-none-any.whl
  Using cached tokenizers-0.9.2.tar.gz (170 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for tokenizers (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> [56 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build
      creating build/lib.macosx-10.9-universal2-cpython-310
      creating build/lib.macosx-10.9-universal2-cpython-310/tokenizers
      copying py_src/tokenizers/__init__.py -> build/lib.macosx-10.9-universal2-cpython-310/tokenizers
      creating build/lib.macosx-10.9-universal2-cpython-310/tokenizers/models
      copying py_src/tokenizers/models/__init__.py -> build/lib

In [2]:
import torch
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from transformers import RobertaForSequenceClassification, RobertaTokenizer, RobertaModel
from transformers import BertForSequenceClassification, BertTokenizer, BertModel
from transformers import AutoModel, AutoTokenizer
from sklearn import metrics
import torch.nn as nn 
import torch.nn.functional as F
from transformers import AdamW
from sklearn.utils import class_weight

2022-12-11 00:56:09.920273: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
n_gpu = torch.cuda.device_count()
#torch.cuda.get_device_name(0)

In [4]:
# Set the maximum sequence length
MAX_LEN = 50

In [5]:
df = pd.read_csv('../bragging_data/bragging_data.csv', header=0, names=['id', 'text', 'sampling', 'round_no', 'label'])

text_training = []
text_testing = []
label_training = []
label_testing = []
labels_number = []
index_training = []
index_testing = []

text_testing_print = []

texts = df.text.values
sampling_ways = df.sampling.values
round_nos = df.round_no.values
labels = df.label.values

for i in labels:
    if i == "not":
        labels_number.append(0)
    else:
        labels_number.append(1)

for i in range(len(sampling_ways)):
    if sampling_ways[i] == 'keyword':
        index_training.append(i)
        #text_training.append(texts[i])
        label_training.append(labels_number[i])
    else:
        index_testing.append(i)
        #text_testing.append(texts[i])
        text_testing_print.append(texts[i])
        label_testing.append(labels_number[i])

In [6]:
texts = ['[CLS] ' + str(sentence) + ' [SEP]' for sentence in texts]

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
tokenized_texts = [tokenizer.tokenize(sent) for sent in texts]

# Use the RoBERTa tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]

# Pad input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype='long', truncating='post', padding='post')
print(input_ids)
# Create attention masks
attention_masks = []
for seq in input_ids:
    seq_mask = [float(i>0) for i in seq]
    attention_masks.append(seq_mask)

[[  101  2079  1050 ...     0     0     0]
 [  101  3057  2024 ...     0     0     0]
 [  101  1045  3342 ...     0     0     0]
 ...
 [  101  1030  5310 ...     0     0     0]
 [  101  1001 15716 ...  3993  2607  2008]
 [  101  2771 12036 ...     0     0     0]]


In [7]:
mask_training = []
mask_testing = []
for i in index_training:
    text_training.append(input_ids[i])
    mask_training.append(attention_masks[i])
for i in index_testing:
    text_testing.append(input_ids[i])
    mask_testing.append(attention_masks[i])

In [20]:
batch_size = 32
n_epoch = 12

x_train = text_training
y_train = label_training


print(len(x_train))
x_dev, x_testing, y_dev, y_testing, mask_dev, mask_testing = train_test_split(text_testing, label_testing, mask_testing, test_size=0.8, random_state=0, stratify=label_testing)

# sklearn
weight_loss = class_weight.compute_class_weight( class_weight ='balanced', classes =  np.unique(y_train), y = y_train)
weight_loss = torch.tensor(weight_loss, dtype=torch.float32).cpu()


# model_roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).cuda()
model_roberta = RobertaForSequenceClassification.from_pretrained('roberta-base', num_labels=2).cpu()


param_optimizer = list(model_roberta.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']        
    
optimizer_grouped_parameters = [
        {'params': [p for n, p in model_roberta.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
        {'params': [p for n, p in model_roberta.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
        ]

optimizer = AdamW(optimizer_grouped_parameters, lr=3e-6)
criterion = nn.CrossEntropyLoss(weight=weight_loss, size_average=None, ignore_index=-100, reduce=None, reduction='mean')

x_train = torch.LongTensor(x_train)
x_dev = torch.LongTensor(x_dev)

y_train = torch.LongTensor(y_train)
y_dev = torch.LongTensor(y_dev)

mask_train = torch.LongTensor(mask_training)
mask_dev = torch.LongTensor(mask_dev)

# Pack to dataLoader
train_data = TensorDataset(x_train, mask_train, y_train)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
    
dev_data = TensorDataset(x_dev, mask_dev, y_dev)
dev_sampler = RandomSampler(dev_data)
dev_dataloader = DataLoader(dev_data, sampler=dev_sampler, batch_size=batch_size) 

# Initialize previous dev loss
previous_valid_loss = 1000

for epoch in range(n_epoch):
    print(epoch)

    # Training
    model_roberta.train()

    train_losses = []
    valid_losses = []

    for step, batch in enumerate(train_dataloader):
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from dataloader
        b_input_ids, b_input_mask, b_labels = batch

        # Clear out the gradients (by default they accumulate)
        optimizer.zero_grad()
        
        # Generate combined representations      
        outputs = model_roberta(input_ids=b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        logits = outputs[1]
        #loss = criterion(logits, b_labels)

        # Backward pass
        loss.backward()

        # track train loss
        train_losses.append(loss.item())

        # Update parameters and take a step using the computed gradient
        optimizer.step()
        
    train_loss = np.average(train_losses)
    print('train loss: {}'.format(train_loss))

    # Validation
    model_roberta.eval()

    predictions = []
    targets = []


    # Evaluate data for one epoch
    for batch in dev_dataloader:
        # Add batch to GPU
        batch = tuple(t.to(device) for t in batch)

        # Unpack the inputs from dataloader
        b_input_ids, b_input_mask, b_labels = batch
      
        with torch.no_grad():

            # Generate combined representations
            outputs = model_roberta(input_ids=b_input_ids, attention_mask=b_input_mask, labels=b_labels)
            loss = outputs[0]
            logits = outputs[1]
            #loss = criterion(logits, b_labels)
          

        valid_losses.append(loss.item())

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        
        labels = b_labels.to('cpu').numpy() 

        predictions = np.append(predictions, np.argmax(logits, axis=1))
        targets = np.append(targets, labels) 

    # Calculate total dev loss 
    valid_loss = np.average(valid_losses)
    print('valid loss: {}'.format(valid_loss))

    # Calculate dev f1 of sverity
    dev_f1 = metrics.f1_score(targets, predictions, average='macro', zero_division=1)
    print("dev_f1:", dev_f1)


    # Save the best model based on dev lossr
    torch.save(model_roberta, './roberta-bragging.pkl')
    print("saved")             
  
x_testing = torch.LongTensor(x_testing)
y_testing = torch.LongTensor(y_testing)
mask_testing = torch.LongTensor(mask_testing)
    
test_data = TensorDataset(x_testing, mask_testing, y_testing) 
test_sampler = SequentialSampler(test_data)
test_dataloader = DataLoader(test_data, sampler=test_sampler, batch_size=batch_size)

# Testing
bragging_model = torch.load('./roberta-bragging.pkl')

test_predictions = []
test_targets = []
bragging_model.eval()

for batch in test_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from dataloader
    b_input_ids, b_input_mask, b_labels = batch
    
    with torch.no_grad():
        # Forward pass, calculate logit predictions
        outputs = bragging_model(input_ids=b_input_ids, attention_mask=b_input_mask)
        logits = outputs[0]
    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    labels = b_labels.to('cpu').numpy()

    test_predictions = np.append(test_predictions, np.argmax(logits, axis=1))
    test_targets = np.append(test_targets, labels)

test_acc = metrics.accuracy_score(test_targets, test_predictions)
test_precision = metrics.precision_score(test_targets, test_predictions, average="macro", zero_division=1)
test_recall = metrics.recall_score(test_targets, test_predictions, average="macro", zero_division=1)
test_f1 = metrics.f1_score(test_targets, test_predictions, average="macro", zero_division=1)
print("test_acc:", test_acc)
print("test_precision:", test_precision)
print("test_recall:", test_recall)
print("test_f1:", test_f1)

target_names = ['class 0', 'class 1']
print(metrics.classification_report(test_targets, test_predictions, target_names=target_names))

3382


ValueError: Found input variables with inconsistent numbers of samples: [3314, 3314, 0, 3314]